# dyPolyChord paper results

This notebook contains the code used to make the Gaussian mixture model results with `dyPolyChord`. For more information see the dynamic nested sampling paper ([Higson et al., 2017](https://arxiv.org/abs/1704.03459)) and the [dyPolyChord module](https://github.com/ejhigson/dyPolyChord).

This notebook requires nested sampling run data which you can generate using ``make_dypolychord_runs.py``; see the module docstring for information about the random seeding used for the numerical results in the paper.

Also required is the ``bsr`` package, which can be downloaded at https://github.com/ejhigson/bsr.

### Select likelihood to run

Must be 'fit' or 'gaussian_mix_4comp_4sep'

In [ ]:
likelihood_name = 'fit'
assert likelihood_name in ['gaussian_mix_4comp_4sep', 'fit']

### Set up imports

In [ ]:
import functools
import numpy as np
import copy
import pandas as pd
import scipy.interpolate
import matplotlib.pyplot as plt
import nestcheck.data_processing
import nestcheck.diagnostics_tables
import nestcheck.estimators as e
import nestcheck.plots
import nestcheck.pandas_functions
import dyPolyChord.output_processing
import dyPolyChord.python_likelihoods as likelihoods
import bsr.plotting
import bsr.data
import bsr.likelihoods
import bsr.basis_functions
import matplotlib
%matplotlib inline
%autoreload 2

# Set plot size, font and fontsize to match LaTeX template
# NB A4 paper is 8.27 × 11.69 inches (=210 × 297 mm)
# STCO
# ----
# Font (=caption font): \OT1/ptm/m/n/10 (= times size 10)
# Footnote font: \OT1/cmr/m/n/10
# Abstract font: \OT1/cmr/m/n/10.95
textwidth = 6.85066 * 0.99  # make 1% smaller to ensure everything fits
textheight = 9.2144 * 0.99  # make 1% smaller to ensure everything fits
colwidth = 3.30719
# Thesis
# ------
# textwidth = 5.1
# Matplotlib setup
# ----------------
fontsize = 8.5  # matplotlib default is 10
matplotlib.rc('text', usetex=True)
matplotlib.rc('font', family='serif', serif='Times New Roman', size=fontsize)

# Get estimator functions
# -----------------------

latex_str_map = {'mathrm{log}': 'log',
                 'std': 'St.Dev.\\',
                 'St.Dev. ': 'St.Dev.\\ ',
                 'mean': 'Mean',
                 'true values': 'Analytic values',
                 'None': ''}

### Get fit likelihood object

In [ ]:
# set up
fit_func = bsr.basis_functions.gg_1d
nfunc = 3
y_error_sigma = 0.07
x_error_sigma= 0.07
data = bsr.data.generate_data(
    fit_func, nfunc, y_error_sigma, x_error_sigma=x_error_sigma)
fit_likelihood_obj = bsr.likelihoods.FittingLikelihood(
    data, fit_func, nfunc, adaptive=False)

### Get estimator function list

In [ ]:
def likelihood_calls(run, logw=None, simulate=False):
    return run['output']['nlike']

def fit_mean(ns_run, logw=None, simulate=False, like=fit_likelihood_obj, x1=0.5):
    if logw is None:
        logw = nestcheck.ns_run_utils.get_logw(ns_run, simulate=simulate)
    w_rel = np.exp(logw - logw.max())
    return like.fit_mean(ns_run['theta'], x1)

estimator_list = [likelihood_calls,
                  e.count_samples,
                  e.logz]

estimator_names = ['likelihood calls'] + [e.get_latex_name(est) for est in estimator_list[1:]]

if likelihood_name == 'gaussian_mix_4comp_4sep':
    estimator_list += [e.param_mean,
                       functools.partial(e.param_mean, param_ind=1),
                       functools.partial(e.param_mean, param_ind=2),
                       functools.partial(e.param_mean, param_ind=3),
                       functools.partial(e.param_mean, param_ind=4),
                       functools.partial(e.param_mean, param_ind=5),
                       e.param_squared_mean,
                       functools.partial(e.param_cred, probability=0.5),
                       functools.partial(e.param_cred, probability=0.84),
                       e.r_mean,
                       functools.partial(e.r_cred, probability=0.5),
                       functools.partial(e.r_cred, probability=0.84)]
    estimator_names = ['likelihood calls'] + [e.get_latex_name(est) for est in estimator_list[3:]]
elif likelihood_name == 'fit':
    estimator_list += [functools.partial(e.param_mean, param_ind=ind)
                       for ind in range(0, fit_likelihood_obj.ndim)]
    estimator_names += [r'$\overline{{{}}}$'.format(name[1:-1])
                        for name in fit_likelihood_obj.get_param_latex_names()]
    for x in np.linspace(0.0, 1.0, 11):
        estimator_list += [functools.partial(fit_mean, x1=x)]
        estimator_names += ['$y({:.1f})$'.format(x)]
estimator_names

### Analytic Results for Gaussian mixture model

#### Convolutions of Gaussians formulae

From [Petersen and Pedersen (2018)](http://www.cim.mcgill.ca/~dudek/417/Papers/matrixOperations.pdf), Section 8.1.8, the product of two Gaussians is

\begin{equation}
\mathcal{N}_x(\mu_1,\Sigma_1) \times \mathcal{N}_x(\mu_2,\Sigma_2) = c_c\mathcal{N}_x(\mu_c,\Sigma_c),
\end{equation}


where

\begin{align}
c_c & = \mathcal{N}_{\mu_1}(\mu_2,(\Sigma_1+\Sigma_2)), \\
\mu_c & = (\Sigma_1^{-1}+\Sigma_2^{-1})^{-1} (\Sigma_1^{-1}\mu_1 + \Sigma_2^{-1}\mu_2), \\
\Sigma_c & = (\Sigma_1^{-1}+\Sigma_2^{-1})^{-1}.
\end{align}

#### Application to the mixture with a Gaussian prior

The posterior for the mixture of 4 unit Gaussians with a Gaussian prior which was used in the paper can be found by convolving the prior with each component to produce a sum of 4 shifted Gaussians.

In our case we have $\Sigma_m = \mathcal{I}$ for the components and $\Sigma_\pi = \mathcal{I} \frac{1}{\sigma_\pi^2}$, $\mu_\pi = \vec{0}$ for the prior. Hence for each component the convolution with the prior is a spherically symmetric Gaussian with

\begin{equation}
\mu_m = \mu_m \frac{\sigma_\pi^2}{1 + \sigma_\pi^2}$.
\end{equation}

Furthermore, for every component

\begin{equation}
\sigma_c = \frac{\sigma_\pi}{\sqrt{1 + \sigma_\pi^2}}
\end{equation}

and

\begin{equation}
c_c = \mathcal{N}_{0}\left(\mu_1,1 + \frac{1}{\sigma_\pi^2}\right),
\end{equation}

which gives spherically symmetric Gaussian with $\sigma = \sqrt{1 + \sigma_\pi^2}$. Furthermore we have $|\mu|=\mathrm{sep}=4$ for all components so $c_c$ is the same for all components and the Bayesian evidence is

\begin{equation}
\mathcal{Z} = c_c.
\end{equation}

In [ ]:
def gaussian_mix_logz(sep=4, dim=10, prior_scale=10):
    rad = np.zeros(dim)
    rad[0] = sep
    const_sigma = np.sqrt(1 + (prior_scale ** 2))
    return likelihoods.Gaussian(sigma=const_sigma)(rad)[0]

def gaussian_mix_means(sep=4, dim=10, prior_scale=10, weights=np.asarray([0.4, 0.3, 0.2, 0.1])):
    positions = [(0, sep), (0, -sep), (sep, 0), (-sep, 0)][:len(weights)]
    factor = prior_scale / np.sqrt(1 + (prior_scale ** 2))
    thetas = np.zeros(dim)
    comp_pos = np.asarray([[0,  4],
                         [0,  -4],
                         [4,  0],
                         [-4, 0]])
    for i in range(2):
        thetas[i] = np.sum(comp_pos[:, i] * weights) * factor
    return thetas


true_values_dict = {e.get_latex_name(e.logz): gaussian_mix_logz()}
param_means = gaussian_mix_means()
for i, val in enumerate(param_means):
    true_values_dict[e.get_latex_name(e.param_mean, param_ind=i)] = val
true_values_dict

### Get Data

In [ ]:
start_run = 0
n_run = 50
# Shared Settings
pc_run_dict = {}
method_names = []
for dynamic_goal in [None, 0, 0.25, 1]:
    if dynamic_goal is None:
        key = 'standard'
    else:
        key = 'dynamic $G=' + str(dynamic_goal) + '$'
    method_names.append(key)
    if likelihood_name == 'gaussian_mix_4comp_4sep':
        root = dyPolyChord.output_processing.settings_root(
            likelihood_name,
            'gaussian',  # prior name
            10,  # dimensions
            dynamic_goal=dynamic_goal,
            nrepeats=50,
            prior_scale=10,
            ninit=200, init_step=100,
            nlive_const=500).replace('.', '_')
    elif likelihood_name == 'fit':
        root = fit_likelihood_obj.get_file_root(1000, 400, dynamic_goal=dynamic_goal)
    print(root)
    pc_run_dict[key] = nestcheck.data_processing.batch_process_data(
        [root + '_' + str(i).zfill(3) for i in range(start_run + 1, start_run + n_run + 1)],
        process_func=nestcheck.data_processing.process_polychord_run,
        func_kwargs={}, errors_to_handle=OSError,
        save_name='cache/' + root + '_' + str(start_run + 1) + '_to_' + str(start_run + n_run),
        load=False, save=True)

### Plot nlive as a function of Log X

In [ ]:
# Calculate logL(logX) and logX(logL) numerically by combining many runs together to get accurate
# logX values for each point logL, then interpolate
comb = nestcheck.ns_run_utils.combine_ns_runs(pc_run_dict['standard'][:100])
logx = nestcheck.ns_run_utils.get_logx(comb['nlive_array'])
logl_given_logx = scipy.interpolate.interp1d(logx, comb['logl'], bounds_error=False, fill_value=-np.inf)
logx_given_logl = scipy.interpolate.interp1d(comb['logl'], logx, bounds_error=False, fill_value=-np.inf)
plot_dict = {}
for method in method_names:
    plot_dict[method] = pc_run_dict[method][:10]

In [ ]:
# Make the plot
if likelihood_name == 'gaussian_mix_4comp_4sep':
    logx_min = -35
    ymax = 3000
elif likelihood_name == 'fit': 
    ymax = 6000
    logx_min = -35
fig = nestcheck.plots.plot_run_nlive(method_names, plot_dict, logx_min=logx_min, ymax=ymax,
                                     logx_given_logl=logx_given_logl,
                                     logl_given_logx=logl_given_logx,
                                     figsize=(textwidth, 2.2))
# For thesis, use left=0.1
adjust_dict = {'left': 0.085, 'right': 0.99, 'bottom': 0.185, 'top': 0.975}
fig.subplots_adjust(**adjust_dict)
fig.savefig('plots/nlive_' + likelihood_name + '.pdf')

### Triangle Plot Diagram

This cell makes an accurate plot of the posterior numerically by combining evenly weighted samples from a lot of nested sampling runs. It requires the `getdist` package, which is available on PyPI and at [https://github.com/cmbant/getdist](https://github.com/cmbant/getdist).

In [ ]:
param_limits = {}
if likelihood_name == 'gaussian_mix_4comp_4sep':
    names = [r'\theta_\hat{' + str(i) + r'}' for i in range(1, 11)]
    for name in names:
        param_limits[name] = [-8, 8]
    names_to_plot = names[:4]
elif likelihood_name == 'fit':
    names = fit_likelihood_obj.get_param_latex_names()
    names = [name[1:-1] for name in names]
    for name in names:
        param_limits[name] = [0, 1]
    names_to_plot = names[:4]
try:
    from getdist import plots, MCSamples
    np.random.seed(0)
    evens = []
    for run in pc_run_dict['dynamic $G=1$']:
        logw = nestcheck.ns_run_utils.get_logw(run)
        w_rel = np.exp(logw - logw.max())
        temp = run['theta'][np.where(w_rel > np.random.random(w_rel.shape))[0], :]
        evens.append(temp)
    samples = MCSamples(samples=np.vstack(evens), names=names, labels=names)
    g = plots.getSubplotPlotter(width_inch=colwidth)
    g.triangle_plot([samples], names_to_plot, shaded=True, param_limits=param_limits)
    # Save as a png as due to the very large number of samples this takes up less memory than a pdf
    g.fig.savefig('plots/triangle_plot.png', dpi=400)
except ImportError:
    print('Install getdist to make this plot')

### Colour contour diagram

In [ ]:
fig = bsr.plotting.plot_runs(
    fit_likelihood_obj,
    pc_run_dict['dynamic $G=1$'][0],
    nfunc_list=[nfunc], plot_data=True,
    figsize=(6.5, 2.5),
    subplot_height=1.8,
    subplot_width=1.8)

### Results Tables

### Efficiency gain DataFrame

In [ ]:
# Generate efficiency gain df
true_values = np.full(len(estimator_names), np.nan)
if likelihood_name == 'gaussian_mix_4comp_4sep':
    include_true_values=True
    include_rmse=True
    for i, name in enumerate(estimator_names):
        try:
            true_values[i] = true_values_dict[name]
        except KeyError:
            pass
elif likelihood_name == 'fit':
    include_true_values=False
    include_rmse=False
method_values = []
for method in method_names:
    method_values.append([nestcheck.ns_run_utils.run_estimators(run, estimator_list)
                          for run in pc_run_dict[method]])
eff_df = nestcheck.pandas_functions.efficiency_gain_df(
    method_names, method_values, estimator_names,
    true_values=true_values, include_true_values=include_true_values,
    include_rmse=include_rmse)
eff_df

In [ ]:
eff_df.xs('std efficiency gain', level='calculation type')[estimator_names[:3] + estimator_names[-11:]]

### Implementation Errors DataFrame

In [ ]:
# diagnostic tests df
error_summary_dict = {}
n_simulate = 10
for method in method_names:
    save_name = ('cache/' + likelihood_name + '_' + method.replace('$', '').replace('=', '_')
                 .replace(' ', '_') + '_' + str(n_simulate) + 'sim')
    print(save_name)
    df_temp = nestcheck.diagnostics_tables.run_list_error_values(
        pc_run_dict[method], estimator_list[1:], estimator_names[1:], n_simulate=n_simulate, parallel=True,
        load=True, save=True, thread_pvalue=False, bs_stat_dist=False, save_name=save_name)
    # error_values_dict[method] = df_temp
    error_summary_dict[method] = nestcheck.diagnostics_tables.error_values_summary(df_temp)
# values_df = pd.concat(error_values_dict)
diag_df = pd.concat(error_summary_dict)

In [ ]:
# show implementation errors
diag_df.xs('implementation std', level='calculation type')# [estimator_names[:3] + estimator_names[-11:]]

### Get paper results tables

In [ ]:
# columns to use in paper
cols = [
    'samples',
    '$\\mathrm{log} \\mathcal{Z}$',
    '$\\overline{\\theta_{\\hat{1}}}$',
    '$\\overline{\\theta_{\\hat{2}}}$',
    '$\\mathrm{median}(\\theta_{\\hat{1}})$',
    '$\\mathrm{C.I.}_{84\\%}(\\theta_{\\hat{1}})$',
    '$\\overline{|\\theta|}$']

In [ ]:
# Paper Table 2
paper_eff_df = nestcheck.pandas_functions.paper_format_efficiency_gain_df(eff_df[cols])
try:
    import texunc
    texunc.print_latex_df(paper_eff_df, min_dp=1, min_dp_no_error=2, str_map=latex_str_map)
except ImportError:
    print('Install texunc to get the results tables in the format used in the paper LaTeX file.')
paper_eff_df

In [ ]:
# Paper Table 3
imp_df = copy.deepcopy(diag_df[cols[1:]].xs('implementation std', level='calculation type', drop_level=False))
row_names = (imp_df.index.get_level_values(1).astype(str) + ' ' +
             imp_df.index.get_level_values(0).astype(str))
row_names = row_names.str.replace('dynamic', '').str.replace('implementation', 'Implementation')
imp_df.index = [row_names, imp_df.index.get_level_values(2)]
try:
    import texunc
    texunc.print_latex_df(imp_df, str_map=latex_str_map)
except ImportError:
    print('Install texunc to get the results tables in the format used in the paper LaTeX file.')
imp_df

In [ ]:
# Paper Table 7
rmse_df = copy.deepcopy(eff_df.loc[:, ~np.isnan(eff_df.loc[('true values', '' , 'value'), :].values)])
row_names = (rmse_df.index.get_level_values(0).astype(str) + ' ' +
             rmse_df.index.get_level_values(1).astype(str))
row_names = row_names.str.replace('dynamic', '').str.replace('rmse', 'RMSE')
rmse_df.index = [row_names, rmse_df.index.get_level_values(2)]
try:
    import texunc
    texunc.print_latex_df(rmse_df, str_map=latex_str_map,  min_dp_no_error=4, max_power=5)
except ImportError:
    print('Install texunc to get the results tables in the format used in the paper LaTeX file.')
rmse_df